In [7]:
# Build base code for making requests to OpenAI API
import requests
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")
model = "claude-sonnet-4-20250514"
url = "https://api.laozhang.ai/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": api_key
}

def add_system_message(messages, content):
    messages.append({"role": "system", "content": content})

def add_user_message(messages, content):
    messages.append({"role": "user", "content": content})

def add_assistant_message(messages, content):
    messages.append({"role": "assistant", "content": content})

def chat(messages,temperature=None):
    params = {
        "model": model,
        "stream": False,
        "messages": messages,
        "max_tokens": 1024,
    }
    if temperature:
        params["temperature"] = temperature

    response = requests.post(url, headers=headers, json=params)
    print(response.json())
    answer = response.json()['choices'][0]['message']['content']
    return answer

In [8]:
system = """
    You are a patient math tutor.
    Do not directly give the answer to the student.
    Instead, guide them to a solution step by step.
"""

messages = []
add_system_message(messages, system)
add_user_message(messages, "What is answer x of the equation 3x+8=20?")
answer = chat(messages)
answer


{'error': {'message': 'user [23957] quota [2003] preConsumedQuota [2123] is not enough', 'localized_message': '用户余额不足, 请充值', 'type': 'shell_api_error', 'param': '', 'code': 'insufficient_user_quota'}}


KeyError: 'choices'

In [ ]:
# system prompt exercise

messages = []

add_user_message(messages, "Write a Python function that checks a string for duplicate characters.")
system = """
    You are a Python engineer who writes very concise code.
"""
add_system_message(messages, system)
answer = chat(messages)   
answer

ProxyError: HTTPSConnectionPool(host='api.laozhang.ai', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ProxyError('Unable to connect to proxy', SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)'))))

In [ ]:
messages = []

add_user_message(messages, "Generate a one sentence movie idea.")
answer = chat(messages, temperature=0.9)
answer

TypeError: chat() got an unexpected keyword argument 'temperature'